In [37]:
from pymongo import MongoClient
import json
import requests
import json
from requests_oauthlib import OAuth1
import base64
import six

## Connect

In [44]:
cfg = json.load(open('config/mongo.json'))

db = 'mongodb://{host}:{port}'.format(**cfg['local'])  # local DB
with MongoClient(db) as client:
    tracksDB = client[cfg['local']['database']]['tracks']
    artistsDB = client[cfg['local']['database']]['artists']

## Insert

In [39]:
config = json.load(open('config/access.json'))
auth_header = base64.b64encode(str(config['Client ID'] + ':' + config['Client Secret']).encode('ascii'))
headers = {
    'Content-Type': 'application/x-www-form-urlencoded',
    'Authorization': 'Basic '+ auth_header.decode('ascii')
}

payload = { 'grant_type': 'client_credentials'}
auth_req = requests.post('https://accounts.spotify.com/api/token', headers=headers, data=payload)

headers = {
    'Authorization': 'Bearer ' + auth_req.json()['access_token']
}

s = requests.Session()
s.headers.update(headers)

In [48]:
artist_ids = ['1Xyo4u8uXC1ZmMpatF05PJ', '2h93pZq0e7k5yf4dywlkpM', '6M2wZ9GZgrQXHCFfjv46we']

headers['Content-Type'] = 'application/json'

payload = {'country': 'NL'}

tracks = []

for artist in artist_ids:
    tracks.extend(s.get('https://api.spotify.com/v1/artists/%s/top-tracks'%(artist), 
                        headers=headers, params=payload).json()['tracks'])

In [51]:
for track in tracks:
    
    tr = s.get('https://api.spotify.com/v1/audio-features/%s'%(track['id']), headers=headers).json()
    tr.update({'name': track['name'], 'popularity': track['popularity'], 'artists': track['artists'][0]['name']})
    tracksDB.insert(tr)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  """


In [50]:
track['artists'][0]['name']

'Dua Lipa'